In [1]:
'''This script prepares the data to create a bubble map that shows 
the change in people living in deserts between 2020 and 2022'''

'This script prepares the data to create a bubble map that shows \nthe change in people living in deserts between 2020 and 2022'

In [2]:
import geopandas as gpd
import pandas as pd

In [3]:
# 2022 data
gdf_2022 = gpd.read_file("../output/finished-geojsons/level_2_results.geojson")
gdf_2022['category'] = gdf_2022.category.str.replace("Desierto Moderado", "Desierto moderado")

In [4]:
def prepare_2020_data():
    
    level_2 = gpd.read_file("../output/finished-geojsons/level_2_results.geojson")
    level_2 = level_2[["parent_name", "name", "code", "parent_code", "geometry"]]
    
    classification_2020 = pd.read_csv("../input/datos-ipys-2020.csv")
    pop_2020 = pd.read_csv("../input/poblacion-cidade-2020.csv", sep="|")
    
    classification_2020 = classification_2020[['Estado', 'Municipio', 'Tipo Desierto 2020']]
    pop_2020 = pop_2020[['Estado', 'Municipio', 'población 2020']]

    df_2020 = pop_2020.merge(classification_2020)
    
    # Manually renames some of the cities in 2020 to match the 2022 names
    df_2020.loc[21, 'Municipio'] = "Carvajal (Valle de Guanape)"
    df_2020.loc[25, 'Municipio'] = "Santa Ana"
    df_2020.loc[61, "Municipio"] = "Obispos"
    df_2020.loc[70, "Municipio"] = "Caroní (Ciudad Guayana)"
    df_2020.loc[71, "Municipio"] = "Padre Pedro Chien (El Palmar)"
    df_2020.loc[118, "Municipio"] = "Palmasola"
    df_2020.loc[152, "Municipio"] = "Alberto Adriani (El Vigía)"
    df_2020.loc[154, "Municipio"] = "Aricagua"
    df_2020.loc[166, "Municipio"] = "Antonio Pinto Salinas (Santa Cruz de Mora)"
    df_2020.loc[167, "Municipio"] = "Pueblo Llano"

        
    # Merges all – the only missing ones will be the Distrito Capital and Vargas entries, which did not exist in 2020
    gdf_2020 = level_2.merge(df_2020, left_on=['parent_name', 'name'], right_on=['Estado', 'Municipio'])
    
    
    # We will create new data entries for DC and Vargas, based on the centroids of the states
    # They will represent the cities of Libertador and La Guaira instead of parroquia
    

    level_1 = gpd.read_file("../output/finished-geojsons/level_1_results.geojson")
    level_1 = level_1[['name', 'geometry']]
    level_1 = level_1[level_1.name.isin(["Distrito capital", "Vargas"])].reset_index()

        
    # Adds data for Libertador (DC) and La Guaira (Vargas)
    gdf_2020 = gdf_2020.append(pop_2020.loc[[333, 334]])
    gdf_2020.loc[ [333, 334], 'Tipo Desierto 2020' ] = 'No desierto'
    
    gdf_2020.loc[333, 'parent_name'] = 'Distrito capital'
    gdf_2020.loc[334, 'parent_name'] = 'Vargas'
    
    gdf_2020.loc[333, 'parent_code'] = 'VE01'
    gdf_2020.loc[334, 'parent_code'] = 'VE24'
    
    gdf_2020.loc[333, 'name'] = 'Libertador'
    gdf_2020.loc[334, 'name'] = 'Vargas (La Guaira)'
    
    gdf_2020.loc[333, 'code'] = 'VE01XX'
    gdf_2020.loc[334, 'code'] = 'VE24XX'
    
    gdf_2020.loc[333, 'población 2020'] = 2090479
    gdf_2020.loc[334, 'población 2020'] = 382123
    
    gdf_2020.loc[333, 'geometry'] = level_1.loc[0, 'geometry']
    gdf_2020.loc[334, 'geometry'] = level_1.loc[1, 'geometry']
    
    
    # Formats
    gdf_2020 = gdf_2020.rename(columns={"población 2020": "population", "Tipo Desierto 2020": "category"})
    gdf_2020 = gdf_2020[['parent_name', 'name', 'code', 'parent_code', 'geometry', 'category', 'population']]
    return gdf_2020



In [6]:
gdf_2020 = prepare_2020_data()

/var/folders/52/vy6xx2q911l7rd368b7ngb8w0000gn/T/ipykernel_8954/4230882905.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf_2020 = gdf_2020.append(pop_2020.loc[[333, 334]])
/Users/menegat/opt/anaconda3/envs/atlas-del-silencio/lib/python3.9/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(


In [7]:
gdf_2020.to_file("map/2020.geojson", driver='GeoJSON')
gdf_2022.to_file("map/2022.geojson", driver='GeoJSON')

In [ ]:
gdf_2022.category.unique()

In [ ]:
df_2020[~df_2020.Municipio.isin(gdf_2020.Municipio)]



In [ ]:
df_2020.loc[21, 'Municipio'] = "Carvajal (Valle de Guanape)"
df_2020.loc[25, 'Municipio'] = "Santa Ana"
df_2020.loc[61, "Municipio"] = "Obispos"
df_2020.loc[70, "Municipio"] = "Caroní (Ciudad Guayana)"
df_2020.loc[118, "Municipio"] = "Palmasola"
df_2020.loc[152, "Municipio"] = "Alberto Adriani (El Vigía)"
df_2020.loc[154, "Municipio"] = "Aricagua"
df_2020.loc[166, "Municipio"] = "Antonio Pinto Salinas (Santa Cruz de Mora)"
df_2020.loc[167, "Municipio"] = "Pueblo Llano"






In [ ]:
gdf_2020.to_json()